# Table of Contents
 <p><div class="lev1"><a href="#Convex-Hull-Tests"><span class="toc-item-num">1&nbsp;&nbsp;</span>Convex Hull Tests</a></div><div class="lev1"><a href="#Min-Area-Rectangle-Tests"><span class="toc-item-num">2&nbsp;&nbsp;</span>Min Area Rectangle Tests</a></div><div class="lev1"><a href="#Diameter-MVBB-Tests"><span class="toc-item-num">3&nbsp;&nbsp;</span>Diameter MVBB Tests</a></div><div class="lev1"><a href="#MVBB-Tests"><span class="toc-item-num">4&nbsp;&nbsp;</span>MVBB Tests</a></div>

In [1]:
import sys,os,imp,re
import math
import numpy as np
import matplotlib as mpl

import matplotlib.cm as cm
import matplotlib.pyplot as plt

import matplotlib.colors as colors
import matplotlib.cm as cmx
from mpl_toolkits.mplot3d import Axes3D


mpl.rcParams['figure.figsize']=(6.0,4.0)    #(6.0,4.0)
mpl.rcParams['font.size']=10                #10 
mpl.rcParams['savefig.dpi']=400             #72 
mpl.rcParams['figure.subplot.bottom']=.1    #.125

plt.rc('font', family='serif')
plt.rc('text', usetex=True)

#inline Shit
%matplotlib inline
%config InlineBackend.figure_format='svg'
%config InlineBackend.rc = {'figure.facecolor': 'white', 'figure.subplot.bottom': 0.125, 'figure.edgecolor': 'white', 'savefig.dpi': 400, 'figure.figsize': (12.0, 8.0), 'font.size': 10}

#GUi shit
%matplotlib tk
mpl.get_configdir()

'/home/zfmgpu/.config/matplotlib'

In [2]:

def plotCube(ax,minP = np.array([-1.0,-1.0,-1.0]), maxP=np.array([1.0,1.0,1.0]),
             trans= np.array([0.0,0.0,0.0]),rotationMatrix=np.diag([1,1,1])):
    from itertools import product, combinations
    r = [-1, 1]
    
    centerPos = (maxP + minP)/2.0;
    #print(centerPos)
    extent = (maxP - minP)/2.0;
    
    points = np.array([(-1, -1, -1),
                     (-1, -1, 1),
                     (-1, 1, -1),
                     (-1, 1, 1),
                     (1, -1, -1),
                     (1, -1, 1),
                     (1, 1, -1),
                     (1, 1, 1)]);
    
    for s, e in combinations(points, 2):
        if np.sum(np.abs(s-e)) == r[1]-r[0]: # no diagonal lines
            p1 = np.array(s,dtype=float); p2 = np.array(e,dtype=float);
            #scale points
            p1*=extent; p2*=extent;
            #rotate and translate points
            p1 = rotationMatrix.dot(p1 + centerPos) + trans;
            p2 = rotationMatrix.dot(p2+centerPos) + trans;
            ax.plot3D(*zip(p1,p2), color="b")

def plotAxis(ax,centerPos,A_IK,plotAxisScale=1):
        for i,c in zip([0,1,2],['r','g','b']):
            I_eK_i = A_IK[:,i];
            lines = list(zip(centerPos,plotAxisScale*I_eK_i+centerPos))
            v = Arrow3D(*lines, mutation_scale=50, lw=1, arrowstyle="-|>", color=c);
            ax.plot3D(*lines, color=c)
            ax.add_artist(v);

def plotAxis2d(ax,centerPos,u,v,plotAxisScale=1):
    
    x = np.vstack((centerPos,plotAxisScale*u+centerPos))
    y = np.vstack((centerPos,plotAxisScale*v+centerPos))
    ax.plot(x.T[0],x.T[1],'r',lw=2)
    ax.plot(y.T[0],y.T[1],'b',lw=2)
        
            
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d
class Arrow3D(FancyArrowPatch):
        def __init__(self, xs, ys, zs, *args, **kwargs):
            FancyArrowPatch.__init__(self, (0,0), (0,0), *args, **kwargs)
            self._verts3d = xs, ys, zs
     
        def draw(self, renderer):
            xs3d, ys3d, zs3d = self._verts3d
            xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
            self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
            FancyArrowPatch.draw(self, renderer)  


In [3]:
def axisEqual3D(ax):
    extents = np.array([getattr(ax, 'get_{}lim'.format(dim))() for dim in 'xyz'])
    sz = extents[:,1] - extents[:,0]
    centers = np.mean(extents, axis=1)
    maxsize = max(abs(sz))
    r = maxsize/2
    for ctr, dim in zip(centers, 'xyz'):
        getattr(ax, 'set_{}lim'.format(dim))(ctr - r, ctr + r)

In [4]:
# get all files
def loadFiles(folderPath,filePathRegex, keyNames):
    files = [ os.path.join(folderPath,f) for f in os.listdir(folderPath) if os.path.isfile( os.path.join(folderPath,f) ) ]

    filePaths=dict();
    for f in files:
        res = re.match(filePathRegex,f)
        if(res):
            key = res.group(1)
            filePaths[key]= dict( [ (keyN, g) for keyN,g in zip(keyNames,res.groups()) ] )
            filePaths[key]["path"] = f
    return filePaths;

In [5]:
import struct
def readPointsMatrixBinary(filePath):
    f = open(filePath, "br")
    # read header (rows,columns)
    (bigEndian,) = struct.unpack("?",f.read(1));
    formatStringBO = "<"; # little endian
    dtype = np.dtype("<f8")
    if(bigEndian):
        formatStringBO = ">";
        dtype = np.dtype(">f8")
    
    (rows,cols,nbytes) = struct.unpack("%sQQQ" % formatStringBO ,  f.read(3*np.int64(0).nbytes))
    print("Matrix Binary: " , rows,cols,nbytes, "big Endian:", bigEndian)                           
    return np.fromfile(f,dtype=dtype);
    

Convex Hull Tests
=================

In [8]:
plt.close("all")

files = loadFiles("./" , ".*ConvexHullTest-(.*?)-(\w+)\.bin", ("name","prec") );
filesOut = loadFiles("./" , ".*ConvexHullTest-(.*?)-(\w+)-Out\.bin", ("name","prec"));

for i,f in files.items():
    
    fOut = filesOut[i]; print(i,f,fOut)
    fig = plt.figure("ConvexHullTest"+str(i),(10,10))
    
    if(i!="PointsRandom14M"):
        points = readPointsMatrixBinary(f["path"]); points=np.reshape(points,(-1,2)) 
        
    hullP =  readPointsMatrixBinary(fOut["path"]); hullP=np.reshape(hullP,(-1,2))
    hullP = np.vstack((hullP,hullP[0]))
    
    plt.plot(hullP.T[0],hullP.T[1],'b-o', ms=20, markerfacecolor='None')
    
    if(i!="PointsRandom14M"):
        plt.scatter(points.T[0],points.T[1],c='r')
        if(len(points)<300):
            nrRange = [(i,p[0],p[1])  for i,p in enumerate(points) ]
            for x in nrRange:      
                plt.annotate('%s' % x[0], xy=x[1:3], textcoords='offset points') # <--
    
#     if(len(hullP)<300):
#         nrRange = [(i,p[0],p[1])  for i,p in enumerate(hullP) ]
#         for x in nrRange:      
#             plt.annotate('%s' % x[0], xy=x[1:3], textcoords='offset points') # <--


PointsBadProjection4 {'name': 'PointsBadProjection4', 'prec': 'double', 'path': './ConvexHullTest-PointsBadProjection4-double.bin'} {'name': 'PointsBadProjection4', 'prec': 'double', 'path': './ConvexHullTest-PointsBadProjection4-double-Out.bin'}
Matrix Binary:  2 16 8 big Endian: False
Matrix Binary:  2 7 8 big Endian: False
Triangle {'name': 'Triangle', 'prec': 'double', 'path': './ConvexHullTest-Triangle-double.bin'} {'name': 'Triangle', 'prec': 'double', 'path': './ConvexHullTest-Triangle-double-Out.bin'}


/opt/python3.4Env/lib/python3.4/site-packages/matplotlib/text.py:2130: UserWarning: You have used the `textcoords` kwarg, but not the `xytext` kwarg.  This can lead to surprising results.
  warnings.warn("You have used the `textcoords` kwarg, but not "


Matrix Binary:  2 3 8 big Endian: False
Matrix Binary:  2 3 8 big Endian: False
PointsOnCricle1000 {'name': 'PointsOnCricle1000', 'prec': 'double', 'path': './ConvexHullTest-PointsOnCricle1000-double.bin'} {'name': 'PointsOnCricle1000', 'prec': 'double', 'path': './ConvexHullTest-PointsOnCricle1000-double-Out.bin'}
Matrix Binary:  2 1000 8 big Endian: False
Matrix Binary:  2 1000 8 big Endian: False
PointsBadProjection {'name': 'PointsBadProjection', 'prec': 'double', 'path': './ConvexHullTest-PointsBadProjection-double.bin'} {'name': 'PointsBadProjection', 'prec': 'double', 'path': './ConvexHullTest-PointsBadProjection-double-Out.bin'}
Matrix Binary:  2 400 8 big Endian: False
Matrix Binary:  2 14 8 big Endian: False
PointsBadProjectionFilter {'name': 'PointsBadProjectionFilter', 'prec': 'double', 'path': './ConvexHullTest-PointsBadProjectionFilter-double.bin'} {'name': 'PointsBadProjectionFilter', 'prec': 'double', 'path': './ConvexHullTest-PointsBadProjectionFilter-double-Out.bin'}


/opt/python3.4Env/lib/python3.4/site-packages/matplotlib/text.py:2130: UserWarning: You have used the `textcoords` kwarg, but not the `xytext` kwarg.  This can lead to surprising results.
  warnings.warn("You have used the `textcoords` kwarg, but not "


Matrix Binary:  2 18 8 big Endian: False
Matrix Binary:  2 8 8 big Endian: False
Point {'name': 'Point', 'prec': 'double', 'path': './ConvexHullTest-Point-double.bin'} {'name': 'Point', 'prec': 'double', 'path': './ConvexHullTest-Point-double-Out.bin'}
Matrix Binary:  2 1 8 big Endian: False
Matrix Binary:  2 1 8 big Endian: False
PointsBadProjection3 {'name': 'PointsBadProjection3', 'prec': 'double', 'path': './ConvexHullTest-PointsBadProjection3-double.bin'} {'name': 'PointsBadProjection3', 'prec': 'double', 'path': './ConvexHullTest-PointsBadProjection3-double-Out.bin'}
Matrix Binary:  2 400 8 big Endian: False
Matrix Binary:  2 8 8 big Endian: False
PointsBadProjection2 {'name': 'PointsBadProjection2', 'prec': 'double', 'path': './ConvexHullTest-PointsBadProjection2-double.bin'} {'name': 'PointsBadProjection2', 'prec': 'double', 'path': './ConvexHullTest-PointsBadProjection2-double-Out.bin'}
Matrix Binary:  2 400 8 big Endian: False
Matrix Binary:  2 10 8 big Endian: False
Line_2 {

Min Area Rectangle Tests
========================

In [8]:
plt.close("all")

files = loadFiles("./" , ".*MinAreaRectangleTest-(.*?)-(\w+).bin", ("name","prec") );
filesOut = loadFiles("./" , ".*MinAreaRectangleTest-(.*?)-(\w+)-Out\.bin", ("name","prec"));


for i,f in files.items():
    if(i in ["NoPoint"]): continue 
        
    fOut = filesOut[i]; print(i,f,fOut)
    fig = plt.figure("MinAreaRectangleTest"+str(i),(10,10))
    ax = fig.gca();
    
    if(i!="PointsRandom10M"):
        points = readPointsMatrixBinary(f["path"]); points=np.reshape(points,(-1,2)) 
    
    rectData =  readPointsMatrixBinary(fOut["path"]); rectData=np.reshape(rectData,(-1,2))
    
    rect = rectData[0:4,]
    rect = np.vstack([rect,rect[0]])
    axis = rectData[4:,]
    print(axis)
    plt.plot(rect.T[0],rect.T[1],'r-', ms=20, markerfacecolor='None')
    
    plotAxis2d(ax,rect[0],axis[0],axis[1]);
    
    if(i!="PointsRandom10M"):
        plt.scatter(points.T[0],points.T[1])
    else:
        plt.scatter(points.T[0][0:400],points.T[1][0:400])
    plt.axis('equal')

Points2DRectFail {'prec': 'double', 'name': 'Points2DRectFail', 'path': './MinAreaRectangleTest-Points2DRectFail-double.bin'} {'prec': 'double', 'name': 'Points2DRectFail', 'path': './MinAreaRectangleTest-Points2DRectFail-double-Out.bin'}
Matrix Binary:  2 18 8 big Endian: False
Matrix Binary:  2 7 8 big Endian: False
[[ 3.38826761 -2.60794988]
 [-0.39487855  0.91873333]
 [-0.91873333 -0.39487855]]
AlmostLine {'prec': 'double', 'name': 'AlmostLine', 'path': './MinAreaRectangleTest-AlmostLine-double.bin'} {'prec': 'double', 'name': 'AlmostLine', 'path': './MinAreaRectangleTest-AlmostLine-double-Out.bin'}
Matrix Binary: 

/opt/python3.4Env/lib/python3.4/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):
/opt/python3.4Env/lib/python3.4/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


 2 4 8 big Endian: False
Matrix Binary:  2 7 8 big Endian: False
[[ 0.          0.        ]
 [ 0.70710678  0.70710678]
 [-0.70710678  0.70710678]]
Point {'prec': 'double', 'name': 'Point', 'path': './MinAreaRectangleTest-Point-double.bin'} {'prec': 'double', 'name': 'Point', 'path': './MinAreaRectangleTest-Point-double-Out.bin'}
Matrix Binary:  2 1 8 big Endian: False
Matrix Binary:  2 7 8 big Endian: False
[[ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]]
TwoPoints {'prec': 'double', 'name': 'TwoPoints', 'path': './MinAreaRectangleTest-TwoPoints-double.bin'} {'prec': 'double', 'name': 'TwoPoints', 'path': './MinAreaRectangleTest-TwoPoints-double-Out.bin'}
Matrix Binary:  2 2 8 big Endian: False
Matrix Binary:  2 7 8 big Endian: False
[[ 1.          0.        ]
 [ 0.5547002   0.83205029]
 [-0.83205029  0.5547002 ]]
PointsRandom10 {'prec': 'double', 'name': 'PointsRandom10', 'path': './MinAreaRectangleTest-PointsRandom10-double.bin'} {'prec': 'double', 'name': 'PointsRandom10', 'path': './MinAreaRectan

Diameter MVBB Tests
========================

In [107]:
plt.close("all")

files = loadFiles("./" , ".*DiameterOOBBTest-(.*?)-(\w+)\.bin", ("name","prec") );
filesOut = loadFiles("./" , ".*DiameterOOBBTest-(.*?)-(\w+)-Out\.txt", ("name","prec") );
filesOut2 = loadFiles("./" , ".*DiameterOOBBTest-(.*?)-(\w+)-Out2\.bin", ("name","prec") );

for i,f in files.items():
        
    fOut = filesOut[i]; fOut2 = filesOut2[i]; print(i,f,fOut,fOut2)
    fig = plt.figure("DiameterTest"+str(i),(10,10))
    
    points = readPointsMatrixBinary(f["path"]); points=np.reshape(points,(-1,3)) 
    OOBB = np.atleast_2d(np.loadtxt(fOut["path"]));
    sampled = readPointsMatrixBinary(fOut2["path"]); sampled=np.reshape(sampled,(-1,3)) 

    K_min = OOBB[0,0:3]
    K_max = OOBB[1,0:3]
    A_IK = OOBB[2:,0:3]
    center = np.zeros((3,));
    #print(A_IK,K_min,K_max,center)

    ax = Axes3D(fig)
    if(i not in ["PointsRandom14M", "Lucy"] ):
        ax.scatter(points.T[0],points.T[1],points.T[2],c='b')
    else:
        plt.scatter(points.T[0][0:2000],points.T[1][0:2000])
    
    ax.scatter(sampled.T[0],sampled.T[1],sampled.T[2],c='r', marker='o', s=200)
    plotCube(ax,K_min,K_max,center,A_IK) # A_IK = R_KI (rotation from I to K)
    plotAxis(ax,center,A_IK,1)
    plotAxis(ax,center,np.identity(3),0.5)
    axisEqual3D(ax)

Plane {'path': './DiameterOOBBTest-Plane-double.bin', 'name': 'Plane', 'prec': 'double'} {'path': './DiameterOOBBTest-Plane-double-Out.txt', 'name': 'Plane', 'prec': 'double'} {'path': './DiameterOOBBTest-Plane-double-Out2.bin', 'name': 'Plane', 'prec': 'double'}
Matrix Binary:  3 3 8 big Endian: False
Matrix Binary:  3 2 8 big Endian: False
UnitCube {'path': './DiameterOOBBTest-UnitCube-double.bin', 'name': 'UnitCube', 'prec': 'double'} {'path': './DiameterOOBBTest-UnitCube-double-Out.txt', 'name': 'UnitCube', 'prec': 'double'} {'path': './DiameterOOBBTest-UnitCube-double-Out2.bin', 'name': 'UnitCube', 'prec': 'double'}
Matrix Binary:  3 8 8 big Endian: False
Matrix Binary:  3 4 8 big Endian: False
PointsSimulationFailMVBB {'path': './DiameterOOBBTest-PointsSimulationFailMVBB-double.bin', 'name': 'PointsSimulationFailMVBB', 'prec': 'double'} {'path': './DiameterOOBBTest-PointsSimulationFailMVBB-double-Out.txt', 'name': 'PointsSimulationFailMVBB', 'prec': 'double'} {'path': './Diameter

/opt/python3.4Env/lib/python3.4/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


MVBB Tests
========================

In [7]:
plt.close("all")

files = loadFiles("./" , ".*MVBBTest-(.*?)-(\w+).bin", ("name","prec") );
filesOut = loadFiles("./" , ".*MVBBTest-(.*?)-(\w+)-Out\.txt", ("name","prec"));


for i,f in files.items():

    fOut = filesOut[i]; 
    print(i,f,fOut);
    fig = plt.figure("MVBBTest" + str(i),(10,10))
    
    
    points = readPointsMatrixBinary(f["path"]); points=np.reshape(points,(-1,3)) 
    OOBB = np.atleast_2d(np.loadtxt(fOut["path"]));

    K_min = OOBB[0,0:3]
    K_max = OOBB[1,0:3]
    A_IK = OOBB[2:5,0:3]
    center = np.zeros((3,));
    print(A_IK,K_min,K_max,center)
    
    ax = Axes3D(fig)
    if(len(points) < 10000):
        ax.scatter(points.T[0],points.T[1],points.T[2],c='b')
    else:
        ax.scatter(points.T[0][0:10000],points.T[1][0:10000],points.T[2][0:10000],c='b')
        
    plotCube(ax,K_min,K_max,center,A_IK) # A_IK = R_KI (rotation from I to K)
    plotAxis(ax,center,A_IK,1)
    plotAxis(ax,center,np.identity(3),0.5)
    axisEqual3D(ax)

PointClouds-Nr-3 {'name': 'PointClouds-Nr-3', 'prec': 'double', 'path': './MVBBTest-PointClouds-Nr-3-double.bin'} {'name': 'PointClouds-Nr-3', 'prec': 'double', 'path': './MVBBTest-PointClouds-Nr-3-double-Out.txt'}
Matrix Binary:  3 2501 8 big Endian: False
[[ 0.70869138 -0.49319411  0.50449588]
 [ 0.63834646  0.14372591 -0.75621205]
 [ 0.3004502   0.85796412  0.41668603]] [-1.42661565 -3.86197932 -1.84620041] [ 5.34794633  0.33660258  5.23716277] [ 0.  0.  0.]
PointClouds-Nr-2 {'name': 'PointClouds-Nr-2', 'prec': 'double', 'path': './MVBBTest-PointClouds-Nr-2-double.bin'} {'name': 'PointClouds-Nr-2', 'prec': 'double', 'path': './MVBBTest-PointClouds-Nr-2-double-Out.txt'}
Matrix Binary:  3 2501 8 big Endian: False
[[ 0.10556901  0.99296013 -0.05371568]
 [ 0.31158544 -0.08432764 -0.94646889]
 [-0.94433559  0.08318076 -0.3182943 ]] [-2.75246443  0.47313016 -5.80166171] [ 4.34356251  3.28634465  1.24852591] [ 0.  0.  0.]
PointClouds-Nr-0 {'name': 'PointClouds-Nr-0', 'prec': 'double', 'pat